# **3a. Formación de vectores con matriz de covarianzas**

**Responsable:**

César Zamora Martínez

**Infraestructura usada:** 
Google Colab, para pruebas


## 0. Instalación de Cupy en Colab

In [1]:
!curl https://colab.chainer.org/install | sh -

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1580  100  1580    0     0  13057      0 --:--:-- --:--:-- --:--:-- 13057
+ apt -y -q install cuda-libraries-dev-10-0
Reading package lists...
Building dependency tree...
Reading state information...
cuda-libraries-dev-10-0 is already the newest version (10.0.130-1).
0 upgraded, 0 newly installed, 0 to remove and 25 not upgraded.
+ pip install -q cupy-cuda100  chainer 
+ set +ex
Installation succeeded!


## 1. Implementación

**Consideraciones:**. Esta etapa supone que se conocen $\mu$ y $\Sigma$ asociados a los activos. El fin de este paso es obtener vectores que serán relevantes para obtener los pesos del portafolio para el inversionista.

Lo cual, se lleva a cabo en concreto a través de las expresiones:

$$ u := \Sigma^{-1} \cdot \mu $$

$$ v := \Sigma^{-1} \cdot 1 $$


En así que a continuación se presenta el código correspondiente:

In [0]:
import cupy as cp

def formar_vectores(mu, Sigma):
  '''
  Calcula las cantidades u = \Sigma^{-1}  \mu y v := \Sigma^{-1} \cdot 1 del problema de Markowitz

  Args:
    mu (cupy array, vector): valores medios esperados de activos (dimension n)
    Sigma (cupy array, matriz): matriz de covarianzas asociada a activos (dimension n x n)

  Return:
    u (cupy array, escalar): vector dado por \cdot Sigma^-1 \cdot mu (dimension n)
    v (cupy array, escalar): vector dado por Sigma^-1 \cdot 1 (dimension n)
  '''

  # Vector auxiliar con entradas igual a 1
  n = Sigma.shape[0]
  ones_vector = cp.ones(n)

  # Formamos vector \cdot Sigma^-1 mu y Sigm^-1 1
  # Nota: 
  #   1) u= Sigma^-1 \cdot mu se obtiene resolviendo  Sigma u = mu
  #   2) v= Sigma^-1 \cdot 1 se obtiene resolviendo  Sigma v = 1

  # Obtiene vectores de interes
  u = cp.linalg.solve(Sigma, mu)
  v = cp.linalg.solve(Sigma, ones_vector)

  return u, v

## 1.1 Matrices de prueba

In [0]:
n= 10

Sigma=cp.random.rand(n, n)
mu=cp.random.rand(n, 1).transpose()[0]

In [17]:
formar_vectores(mu, Sigma)

(array([ 1.70436749, -0.82725263,  0.60542372,  0.50318876,  0.79590507,
         1.60989746, -1.17741502, -0.24717945, -0.41655096, -0.81374065]),
 array([ 0.72241573, -0.56358909,  1.35084842,  0.30239069, -0.31384783,
         0.69895346, -1.57373494, -0.07376877,  0.92177503,  0.69078127]))